# Generate clean_primary_data.pk
**Output:** `../../data/intermediate_data_02/clean_primary_data.pk`

In [39]:
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
primary_data = pd.read_excel("../../data/raw_data/Summary_of_primary_data.xlsx", 'Sheet1')
primary_data[:3]

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,Genus,Species,First_author,Publication_year,Type_of_study,Genes_sampled,Genes_manipulated,Genes_with_effect,Genes_no_effect
1,Chlamydomonas,reinhardtii,Lv,2013,knock_out; transcriptomics,NaN,Cre05.g248150.t1; Cre06.g299050.t1.2,Cre06.g299050.t1.2,Cre05.g248150.t1
5,Chlamydomonas,reinhardtii,Miller,2010,transcriptomics,NaN,NaN,NaN,NaN
7,Chlamydomonas,reinhardtii,Kwak,2017,transcriptomics,NaN,NaN,NaN,NaN


In [16]:
primary_data = primary_data[['Genus', 'Species','First_author','Publication_year',\
                             'Type_of_study','Genes_sampled','Genes_manipulated', 'Genes_with_effect',\
                             "Genes_no_effect"]]
subset = primary_data[primary_data.Genus=='Chlamydomonas'][primary_data.Species=='reinhardtii'].reset_index()
subset[:3]

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,index,Genus,Species,First_author,Publication_year,Type_of_study,Genes_sampled,Genes_manipulated,Genes_with_effect,Genes_no_effect
0,1,Chlamydomonas,reinhardtii,Lv,2013,knock_out; transcriptomics,NaN,Cre05.g248150.t1; Cre06.g299050.t1.2,Cre06.g299050.t1.2,Cre05.g248150.t1
1,5,Chlamydomonas,reinhardtii,Miller,2010,transcriptomics,NaN,NaN,NaN,NaN
2,7,Chlamydomonas,reinhardtii,Kwak,2017,transcriptomics,NaN,NaN,NaN,NaN


In [31]:
df = pd.DataFrame()
subset.Publication_year = subset.Publication_year.astype(str)
df['source'] = subset.First_author+"_"+subset.Publication_year
df['manipulated_gene'] = subset.apply(lambda x: 1 if x.Genes_manipulated is not np.nan else 0, axis =1)
df['type_of_study']  = subset.apply(lambda x: x.Type_of_study.split("; "), axis =1)

In [34]:
df[:3]

,source,manipulated_gene,type_of_study
0,Lv_2013,1,"[knock_out, transcriptomics]"
1,Miller_2010,0,[transcriptomics]
2,Kwak_2017,0,[transcriptomics]


In [40]:
with open('../../data/dfe_alpha/clean_primary_data.pk', 'wb') as f:
    pk.dump(df, f)

In [37]:
types_of_study = list(np.unique([i for sublist in df.type_of_study for i in sublist]))
print(types_of_study)

['RT_qPCR', 'gene_silence', 'genetic_transformation', 'genomics', 'knock_out', 'proteomics', 'transcriptomics']
